In [1]:
!pip install python-telegram-bot==5.3.0

In [3]:
import tensorflow as tf
import sys
import os
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
import logging

print("Librerías importadas correctamente")
def classify(image_path):
    # Read the image_data
    image_data = tf.io.gfile.GFile(image_path, 'rb').read()

    # Loads label file, strips off carriage return
    label_lines = [line.rstrip() for line in tf.io.gfile.GFile("tf_files/retrained_labels.txt")]

    # Unpersists graph from file
    with tf.io.gfile.GFile("tf_files/retrained_graph.pb", 'rb') as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

    with tf.compat.v1.Session() as sess:
        # Feed the image_data as input to the graph and get first prediction
        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
        
        predictions = sess.run(softmax_tensor, \
                {'DecodeJpeg/contents:0': image_data})
        
        # Sort to show labels of first prediction in order of confidence
        top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]
        
        result = None
        for node_id in top_k:
            human_string = label_lines[node_id]
            accuracy = predictions[0][node_id]
            print('%s (score = %.5f)' % (human_string, accuracy))
            if accuracy >= 0.9:
                result = human_string
    return result

print("Método para clasificar creado correctamente")

Librerías importadas correctamente
Método para clasificar creado correctamente


In [4]:
# ClassifyImagesBot
token = '931496766:AAFe8v3MkmMkeS6OJFXPfAOKnVdYBb_awJg'

print("token registrado")

token registrado


In [5]:
try:
    logging.basicConfig(
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
    logger = logging.getLogger(__name__)
except Exception as e:
    print("Error logging {}".format(e.args))

def start(bot, update):
    try:
        username = update.message.from_user.username
        message = "Hello " + username
        update.message.reply_text(message)
    except Exception as e:
        print("Error start {}".format(e.args))


def help(bot, update):
    try:
        username = update.message.from_user.username
        update.message.reply_text('Hello {}, please send a image for classify'.format(username))
    except Exception as e:
        print("Error help {}".format(e.args))

def analize(bot, update):
    try:
        message = "Receiving image..."
        update.message.reply_text(message)
        print(message)
        
        photo_file = bot.getFile(update.message.photo[-1].file_id)
        id_user = update.message.from_user.id
        id_file = photo_file.file_id
        id_analisis = str(id_user) + "-" + str(id_file)
        
        filename = os.path.join('downloads/', '{}.jpg'.format(id_analisis))
        photo_file.download(filename)
        message = "Image received, analyzing, please wait a few seconds"
        update.message.reply_text(message)
        print(message)
        
        result = classify(filename)
        print(result)
        update.message.reply_text(result)
        print("Waiting image..")
    except Exception as e:
        print("Error analize {}".format(e.args))


def echo(bot, update):
    try:
        update.message.reply_text(update.message.text)
        print("Receiving text...")
        print("Waiting for another test...")
        print(update.message.from_user)
    except Exception as e:
        print("Error echo {}".format(e.args))

def error(bot, update, error):
    try:
        logger.warn('Update "%s" caused error "%s"' % (update, error))
    except Exception as e:
        print("Error error {}".format(e.args))

def main():
    try:
        print('ClassifyImagesBot init token')

        updater = Updater(token)
        # Get the dispatcher to register handlers
        dp = updater.dispatcher

        print('ClassifyImagesBot init dispatcher')


        # on different commands - answer in Telegram
        dp.add_handler(CommandHandler("start", start))
        dp.add_handler(CommandHandler("help", help))

        # on noncommand detect the document type on Telegram
        dp.add_handler(MessageHandler(Filters.text, echo))
        dp.add_handler(MessageHandler(Filters.photo, analize))

        # log all errors
        dp.add_error_handler(error)

        # Start the Bot
        updater.start_polling()
        print('ClassifyImagesBot ready')
        updater.idle()
    except Exception as e:
        print("Error main {}".format(e.message))

print("Bot configurado correctamente")

Bot configurado correctamente


In [ ]:
if __name__ == '__main__':
    try:
        main()
    except Exception as e:
        print("Error name: {}".format(e.args))

ClassifyImagesBot init token
ClassifyImagesBot init dispatcher
ClassifyImagesBot ready
Receiving text...
Waiting for another test...
{'id': 889204612, 'first_name': 'Ser', 'type': '', 'last_name': 'R', 'username': ''}
Receiving image...
Error analize (2, 'No such file or directory')
Receiving image...
Image received, analyzing, please wait a few seconds
mouse (score = 0.99236)
keyboard (score = 0.00764)
mouse
Waiting image..


2020-02-01 02:08:59,859 - telegram.ext.updater - ERROR - Error while getting Updates: Timed out
C:\Users\sergio\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
2020-02-01 02:08:59,869 - __main__ - WARNING - Update "None" caused error "Timed out"
2020-02-01 05:23:53,541 - telegram.ext.updater - ERROR - Error while getting Updates: Timed out
2020-02-01 05:23:53,541 - __main__ - WARNING - Update "None" caused error "Timed out"
2020-02-01 05:23:54,546 - urllib3.connectionpool - WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000002AC0E293B08>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /bot931496766:AAFe8v3MkmMkeS6OJFXPfAOKnVdYBb_awJg/getUpdates
2020-02-01 05:23:54,546 - urllib3.connectionpool - WARNING - Retrying (Retry(total=1, connec